# Turbine operation flags

This example demonstrates how to run a wind farm simulation with given operational flags, that indicate if turbines are switched on or off at each state. There is more than one way for achieving this within `foxes`, but here we will make use of the `OpFlagController` which was specifically added for this purpose.

These are the imports that we will need:

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

import foxes
import foxes.variables as FV
import foxes.constants as FC

This example uses the default engine:

In [ ]:
engine = foxes.Engine.new("default")

We first generate the input data, starting with three simple wind states:

In [ ]:
sdata = pd.DataFrame(
    {
        FV.WS: [10, 10, 10],
        FV.WD: [270, 270, 90],
    }
)
sdata.index.name = FC.STATE
sdata

The spatially uniform states based on this data are created by

In [ ]:
states = foxes.input.states.StatesTable(
    sdata,
    output_vars=[FV.WS, FV.WD, FV.TI, FV.RHO],
    fixed_vars={FV.TI: 0.05, FV.RHO: 1.225},
)

We consider a simple row of turbines as our wind farm:

In [ ]:
farm = foxes.WindFarm()
foxes.input.farm_layout.add_row(
    farm=farm,
    xy_base=[0.0, 0.0],
    xy_step=[800.0, 0.0],
    n_turbines=3,
    turbine_models=["DTU10MW"],
    verbosity=0,
)

Now let's create the operational flag for the 3 states and 3 turbines:

In [ ]:
odata = xr.Dataset(
    {
        FV.OPERATING: (
            (FC.STATE, FC.TURBINE),
            np.array(
                [
                    [1, 1, 1],
                    [0, 1, 1],
                    [0, 0, 1],
                ],
                dtype=bool,
            ),
        )
    }
)
odata[FV.OPERATING].values

We now create the `OpFlagController` based on this data, and select it when creating the algorithm object:

In [ ]:
mbook = foxes.ModelBook()
mbook.farm_controllers["op_cntrl"] = foxes.models.farm_controllers.OpFlagController(
    odata
)

In [ ]:
algo = foxes.algorithms.Downwind(
    farm,
    states,
    wake_models=["Bastankhah2014_linear_lim_k004"],
    farm_controller="op_cntrl",
    mbook=mbook,
)

We can now calculate the farm results:

In [ ]:
with engine:
    farm_results = algo.calc_farm()
farm_results.to_dataframe()[[FV.WD, FV.AMB_REWS, FV.REWS, FV.CT, FV.P, FV.OPERATING]]

Indeed the power and ct values are zero for non-operating turbines. In fact the turbine type and all other turbine models are not evaluated at all for those turbines. The non-operating values for the variables are zero for `FV.P` and `FV.CT` and `np.nan` for the rest. This choices can be overloaded by passing a dictionary for the parameter `non_op_values` in the constructor of the `OpFlagController` class.

One way of visualizing the power pattern for all states and turbines is the `StateTurbineMap`, indicating the zero power results for the switched-off turbines:

In [ ]:
o = foxes.output.StateTurbineMap(farm_results)
ax = o.plot_map(FV.P, cbar_label="Power [kW]")

Finally, let's look at the flow fields of the three states. The non-operating turbines do not contribute to the wakes:

In [ ]:
with engine:
    o = foxes.output.FlowPlots2D(algo, farm_results)
    plot_data = o.get_states_data_xy(
        FV.WS, resolution=10, xmin=-500, xmax=2500, verbosity=0
    )

for fig in o.gen_states_fig_xy(plot_data):
    plt.show()